In [1]:
!python -V

Python 3.8.3


In [3]:
from allrecipes import AllRecipes

In [4]:
from bs4 import BeautifulSoup

import urllib.parse
import urllib.request

import re
import ssl

In [5]:
cuisine = ['Mexican', 'Middle-Eastern', 'French', 'Thai', 'Chinese', 'Japanese', 'Italian', 'Peruvian']

In [9]:
def main(c):
    ing = []
    direct = []
    urls = []
    
    for i in c:
        print(i)
        u,v, links=process_recipes(i)
        ing.append(u)
        direct.append(v)
        urls.append(links)
        
        u2,v2, links2 = scrape_additional(i)
        for s in u2:
            ing.append(s)
        for s in v2:
            direct.append(s)
        for s in links2:
            urls.append(s)
        
    return ing, direct, urls

def process_recipes(search):
    query = AllRecipes.search(search)
    ins, uris = scrape_ingredients(query)
    STEPS = scrape_steps(query)
    
    return ins, STEPS, uris

def scrape_additional(search_string):
    base_url = "https://allrecipes.com/search?"
    query_url = urllib.parse.urlencode({"q": search_string})
    url = base_url + query_url
    req = urllib.request.Request(url)
    req.add_header('Cookie', 'euConsent=true')
    handler = urllib.request.HTTPSHandler(context=ssl._create_unverified_context())
    opener = urllib.request.build_opener(handler)
    response = opener.open(req)
    html_content = response.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    list_ins = []
    list_steps = []
    list_links = []
    for a in soup.findAll(class_='pagination__item')[1:]:
        l=process_additional_pages(a.find("a")['href'])
        ins, u2 = scrape_ingredients(l)
        list_ins.append(ins)
        list_links.append(u2)
        directions = scrape_steps(l)
        list_steps.append(directions)
    return list_ins, list_steps, list_links
    
def scrape_ingredients(q):
    ingredients = {}
    u = {}
    for m in q:
        main_recipe_url = m['url']
        u[m['name']] = main_recipe_url
        detailed_recipe = AllRecipes.get(main_recipe_url)
        goods = []
        for j in detailed_recipe['ingredients']:
            goods.append(j)
        ingredients[m['name']] = goods
    return ingredients, u

def scrape_steps(q):
    steps = {}
    for m in q:
        main_recipe_url = m['url']
        detailed_recipe = AllRecipes.get(main_recipe_url)
        st = []
        for j in detailed_recipe['steps']:
            st.append(j)
        steps[m['name']] = st
    return steps

In [10]:
def process_additional_pages(link):
    req= urllib.request.Request(link)
    req.add_header('Cookie', 'euConsent=true')
    handler = urllib.request.HTTPSHandler(context=ssl._create_unverified_context())
    opener = urllib.request.build_opener(handler)
    response = opener.open(req)
    html_content = response.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    search_data = []
    articles = soup.findAll("a", {"class": "mntl-card-list-items"})
    articles = [a for a in articles if a["href"].startswith("https://www.allrecipes.com/recipe/")]
    for article in articles:
        data = {}
        try:
            data["name"] = article.find("span", {"class": "card__title"}).get_text().strip(' \t\n\r')
            data["url"] = article['href']
            try:
                data["rate"] = len(article.find_all("svg", {"class": "icon-star"}))
                try:
                    if len(article.find_all("svg", {"class": "icon-star-half"})):
                        data["rate"] += 0.5
                except Exception:
                    pass
            except Exception as e0:
                data["rate"] = None
            try:
                data["image"] = article.find('img')['data-src']
            except Exception as e1:
                try:
                    data["image"] = article.find('img')['src']
                except Exception as e1:
                    pass
                if "image" not in data:
                    data["image"] = None
        except Exception as e2:
            pass
        if data:
            search_data.append(data)
    return search_data

In [11]:
ingredients, how_to, full_urls = main(cuisine)

Mexican
Middle-Eastern
French
Thai
Chinese
Japanese
Italian
Peruvian


In [12]:
import unicodedata

In [13]:
pattern = r"\d+(?:/\d+)?(?:\s*\d+)?\s*(?:\w+)?|\([^)]*\)|,\s*\w+"
for curr in ingredients:
    for burr in curr.keys():
        for durr in range(len(curr[burr])):
            curr[burr][durr] = re.sub(pattern, '', unicodedata.normalize("NFKD", re.sub(pattern, '', curr[burr][durr])).encode("ascii", "ignore").decode("utf-8"))
            

In [14]:
"""
\d+ = matches >=1 digit
(?:/\d+)? = matches an optional fraction pattern
(?:\s*\d+)? = matches a pattern that consist of whitespace and >=1 number(s)
\s*(?:\w+)? = matches a pattern of one or more whitespace character and optional one or more words
|\([^)]*\) = matches a pattern of words inside a parenthesis, including the parenthesis
|,\s*\w+ = matches a pattern of comma followed by one or more whitespace characters and one or more words
"""

'\n\\d+ = matches >=1 digit\n(?:/\\d+)? = matches an optional fraction pattern\n(?:\\s*\\d+)? = matches a pattern that consist of whitespace and >=1 number(s)\n\\s*(?:\\w+)? = matches a pattern of one or more whitespace character and optional one or more words\n|\\([^)]*\\) = matches a pattern of words inside a parenthesis, including the parenthesis\n|,\\s*\\w+ = matches a pattern of comma followed by one or more whitespace characters and one or more words\n'

In [15]:
pattern = r"\d+(?:/\d+)?(?:\s*\d+)?\s*(?:\w+)?|\([^)]*\)|\n.*"
for curr in how_to:
    for burr in curr.keys():
        for durr in range(len(curr[burr])):
            curr[burr][durr] = re.sub(pattern, '', unicodedata.normalize("NFKD", re.sub(pattern, '', curr[burr][durr])).encode("ascii", "ignore").decode("utf-8"))
       

In [16]:
# turn list of ingredients into one sentence
ing_sentence = {}
for curr in ingredients:
    for burr in curr.keys():
        ing_sentence[burr] = ''.join(curr[burr])

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [17]:
sentence_embeddings = model.encode(list(ing_sentence.values()))

In [18]:
cosine_sim_matrix_bert = cosine_similarity(sentence_embeddings)

In [19]:
import pandas as pd

In [20]:
cosine_sim_df_bert = pd.DataFrame(cosine_sim_matrix_bert, index=ing_sentence.keys(), columns = ing_sentence.keys())

In [21]:
cosine_sim_df_bert

,Pinto Beans With Mexican-Style Seasonings,Mexican Corn on the Cob (Elote),Authentic Mexican Shrimp Cocktail (Coctel de Camarones estilo Mexicano),Mexican Wedding Cookies,Mexican Pizza,Mexican Rice with Bell Pepper,Mexican Tinga,Mexican Whole Wheat Flour Tortillas,Vegetarian Mexican Inspired Stuffed Peppers,Fantastic Mexican Dip,...,Mexican Drunken Beans,Papa a la Huancaína (Huancayo-Style Potatoes),Grilled Pollo a la Brasa,Pisco Sour Cocktail,Lemon Pisco Sour,Mango Ceviche,Peruvian Aji Verde,Vegetarian Huevo Saltado,Chupe (Peruvian Shrimp Chowder),Peruvian Mashed Beans
Pinto Beans With Mexican-Style Seasonings,1.000000,0.831933,0.905415,0.714414,0.929212,0.919121,0.921246,0.695168,0.930917,0.881169,...,0.913980,0.940912,0.925814,0.616236,0.706235,0.883777,0.886400,0.942859,0.912446,0.889621
Mexican Corn on the Cob (Elote),0.831933,1.000000,0.750170,0.792051,0.864751,0.758092,0.754357,0.803032,0.800306,0.852993,...,0.843177,0.825686,0.824995,0.742773,0.744388,0.799379,0.723485,0.802568,0.841798,0.718619
Authentic Mexican Shrimp Cocktail (Coctel de Camarones estilo Mexicano),0.905415,0.750170,1.000000,0.687084,0.873399,0.881446,0.873287,0.664046,0.914333,0.819124,...,0.875660,0.906308,0.890322,0.660315,0.698591,0.853470,0.849677,0.886048,0.850735,0.840562
Mexican Wedding Cookies,0.714414,0.792051,0.687084,1.000000,0.692828,0.653901,0.581858,0.836401,0.668235,0.689888,...,0.672906,0.728621,0.747155,0.801408,0.824335,0.653086,0.703305,0.721816,0.746094,0.647551
Mexican Pizza,0.929212,0.864751,0.873399,0.692828,1.000000,0.913336,0.903057,0.715953,0.944075,0.948404,...,0.943062,0.937932,0.919108,0.636628,0.652463,0.888915,0.816571,0.899430,0.910555,0.836603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mango Ceviche,0.883777,0.799379,0.853470,0.653086,0.888915,0.861737,0.849516,0.649593,0.862809,0.809624,...,0.835581,0.884040,0.891364,0.641979,0.706764,1.000000,0.841300,0.853791,0.801026,0.781287
Peruvian Aji Verde,0.886400,0.723485,0.849677,0.703305,0.816571,0.839193,0.832746,0.599382,0.818186,0.788921,...,0.789750,0.875465,0.850216,0.628817,0.733086,0.841300,1.000000,0.864208,0.790467,0.865498
Vegetarian Huevo Saltado,0.942859,0.802568,0.886048,0.721816,0.899430,0.920798,0.909586,0.705500,0.885169,0.874125,...,0.903977,0.923068,0.896741,0.599171,0.685422,0.853791,0.864208,1.000000,0.919606,0.853544
Chupe (Peruvian Shrimp Chowder),0.912446,0.841798,0.850735,0.746094,0.910555,0.913881,0.882259,0.736927,0.895458,0.892269,...,0.926818,0.903139,0.904443,0.614758,0.681598,0.801026,0.790467,0.919606,1.000000,0.861288


In [22]:
def get_top_similar(row, top = 5):
    top_similar = row.drop(row.name)
    return top_similar.nlargest(top).index.tolist()

In [23]:
top_similar_dict_bert = {}
for curr in cosine_sim_df_bert.index:
    top_similar_dict_bert[curr] = get_top_similar(cosine_sim_df_bert.loc[curr])

top_similar_df_bert = pd.DataFrame.from_dict(top_similar_dict_bert, orient='index', columns=[f"Similar Recipe {i}" for i in range(1, 6)]) 

In [24]:
top_similar_df_bert

,Similar Recipe 1,Similar Recipe 2,Similar Recipe 3,Similar Recipe 4,Similar Recipe 5
Pinto Beans With Mexican-Style Seasonings,Peruvian Lomo Saltado,Ash Reshteh (Persian Noodle and Bean Soup),Peruvian Potato-Chicken Salad (Causa Rellena),Huancaina Sauce,Spicy One-Pot Shakshouka
Mexican Corn on the Cob (Elote),Mexican Style Cornbread,Absolute Mexican Cornbread,Rich Mexican Corn,5-Ingredient Mexican Casserole,Japanese Shrimp Sauce
Authentic Mexican Shrimp Cocktail (Coctel de Camarones estilo Mexicano),Mexican Shrimp Cocktail,Thai Shrimp Curry with a Kick,Easy Baked Falafel with Cucumber-Yogurt Sauce,Thai Beef Salad,Roast Peruvian Turkey
Mexican Wedding Cookies,French Breakfast Puffs,Almond French Toast,Choereg (Armenian Easter Bread),Chocolate Italian Cream Cake,Peruvian Alfajores with Manjar Blanco
Mexican Pizza,Boris' Mexican Rice,Stuffed Mexican Peppers,Easy Mexican Casserole,Mexican Quesadilla Casserole,Italian Spaghetti Sauce with Meatballs
...,...,...,...,...,...
Mango Ceviche,Pittsburgh Football Party Cilantro Salsa,Mexican Beans,French Onion Dip From Scratch,Hot Italian Giardiniera,Easy Shakshuka
Peruvian Aji Verde,Trishie's Chinese-Style Salad Dressing,Huancaina Sauce,Wasabi and Soy Sauce Hummus,Japanese Influenced BBQ Sauce,Mujadarra (Lentils with Rice)
Vegetarian Huevo Saltado,Pasta with Italian Sausage and Pumpkin Sauce,Italian Sausage and Gnocchi Soup,Bechamel Chicken Pasta,Italian Stuffed Chicken Breast,Quick Italian Vegetable Soup
Chupe (Peruvian Shrimp Chowder),Japanese-Style Deep Fried Chicken,Zucchini Noodles Pad Thai,Joe's Fusion Chicken Pad Thai,Bechamel Chicken Pasta,Thai Peanut Chicken


In [27]:
uris = {}
for curr in full_urls:
    for burr in curr.keys():
        uris[burr] = curr[burr]

In [29]:
def make_hyperlink(url, text):
    return f'<a href="{url}" target="_blank">{text}</a>'

# Create hyperlinks in the 'URL' column using the 'Recipe Name' column and the recipe_dict
top_similar_df_bert['URL #1'] = top_similar_df_bert['Similar Recipe 1'].apply(lambda name: make_hyperlink(uris.get(name, ''), name))
top_similar_df_bert['URL #2'] = top_similar_df_bert['Similar Recipe 2'].apply(lambda name: make_hyperlink(uris.get(name, ''), name))
top_similar_df_bert['URL #3'] = top_similar_df_bert['Similar Recipe 3'].apply(lambda name: make_hyperlink(uris.get(name, ''), name))
top_similar_df_bert['URL #4'] = top_similar_df_bert['Similar Recipe 4'].apply(lambda name: make_hyperlink(uris.get(name, ''), name))
top_similar_df_bert['URL #5'] = top_similar_df_bert['Similar Recipe 5'].apply(lambda name: make_hyperlink(uris.get(name, ''), name))


In [64]:
top_similar_df_bert.to_excel('data_recipes_similarity.xlsx')

In [54]:
#vectorizer = CountVectorizer()
#sentence_vectors = vectorizer.fit_transform(ing_sentence.values())
#cosine_sim_matrix = cosine_similarity(sentence_vectors)

In [55]:
#cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=ing_sentence.keys(), columns = ing_sentence.keys())

In [56]:
#top_similar_dict = {}
#for curr in cosine_sim_df.index:
#    top_similar_dict[curr] = get_top_similar(cosine_sim_df.loc[curr])

#top_similar_df = pd.DataFrame.from_dict(top_similar_dict, orient='index', columns=[f"Similar Recipe {i}" for i in range(1, 6)])

In [57]:
#top_similar_df

In [ ]:
# structure of data 
# think about deliverables
# 1. what cuisines can be clustered based on usage of spice
# 2. what associations can be made between cuisines 1st and 2nd ingredients (fusion)
# 3. personal project: coffee and menu design 